In [1]:
# %%
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Import RandomForest
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [2]:
# Initialize MediaPipe Face Landmarker
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# Initialize MediaPipe drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define function to compute Euclidean distance in 3D
def distance_3d(p1, p2):
    return np.sqrt(np.sum((np.array(p1) - np.array(p2)) ** 2))


def draw_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)

    for idx in range(len(face_landmarks_list)):
        face_landmarks = face_landmarks_list[idx]

        # Create landmark proto
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])

        # Draw face landmarks
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

    return annotated_image


In [3]:
def extract_features(coords):
    # Define indices for landmarks
    landmark_indices = {
        'forehead': 10,
        'chin': 152,
        'left_cheek': 234,
        'right_cheek': 454,
        'left_eye': 263,
        'right_eye': 33,
        'nose_tip': 1
    }

    # Extract features based on landmark indices
    features = []
    landmarks_dict = {name: coords[idx] for name, idx in landmark_indices.items()}

    # Calculate distances between important landmarks
    features.append(distance_3d(landmarks_dict['forehead'], landmarks_dict['chin']))  # Face height
    features.append(distance_3d(landmarks_dict['left_cheek'], landmarks_dict['right_cheek']))  # Face width
    features.append(distance_3d(landmarks_dict['left_eye'], landmarks_dict['right_eye']))  # Eye distance

    # Additional distances
    features.append(distance_3d(landmarks_dict['nose_tip'], landmarks_dict['left_eye']))  # Nose to left eye
    features.append(distance_3d(landmarks_dict['nose_tip'], landmarks_dict['right_eye']))  # Nose to right eye
    features.append(distance_3d(landmarks_dict['chin'], landmarks_dict['left_cheek']))  # Chin to left cheek
    features.append(distance_3d(landmarks_dict['chin'], landmarks_dict['right_cheek']))  # Chin to right cheek
    features.append(distance_3d(landmarks_dict['forehead'], landmarks_dict['left_eye']))  # Forehead to left eye
    features.append(distance_3d(landmarks_dict['forehead'], landmarks_dict['right_eye']))  # Forehead to right eye

    # # Additional features

    # # Facial aspect ratios
    # face_width = distance_3d(landmarks_dict['left_cheek'], landmarks_dict['right_cheek'])
    # face_height = distance_3d(landmarks_dict['forehead'], landmarks_dict['chin'])
    # eye_distance = distance_3d(landmarks_dict['left_eye'], landmarks_dict['right_eye'])

    # features.append(face_width / face_height)  # Aspect ratio of face width to height
    # features.append(face_height / eye_distance)  # Aspect ratio of face height to eye distance

    # # More distance features
    # features.append(distance_3d(landmarks_dict['left_eye'], landmarks_dict['chin']))  # Eye to chin
    # features.append(distance_3d(landmarks_dict['right_eye'], landmarks_dict['chin']))  # Eye to chin
    # features.append(distance_3d(landmarks_dict['left_cheek'], landmarks_dict['forehead']))  # Cheek to forehead
    # features.append(distance_3d(landmarks_dict['right_cheek'], landmarks_dict['forehead']))  # Cheek to forehead

    return np.array(features)

In [4]:
def process_data(folder_path):
    faces = []
    labels = []
    
    # Process each image in the dataset
    for shape in os.listdir(folder_path):
        print(f"Processing face shape category: {shape}")
        k = shape
        shape_pth = os.path.join(folder_path, shape)

        for img in os.listdir(shape_pth):
            img_pth = os.path.join(shape_pth, img)

            # Load the image
            pic = cv2.imread(img_pth)

            if pic is None:
                print(f"Error loading image {img_pth}. Skipping...")
                continue

            # Convert the image to RGB as required by MediaPipe
            rgb_image = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)

            # Create a MediaPipe Image object
            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)

            # Detect face landmarks
            detection_result = detector.detect(image)

            if not detection_result.face_landmarks:
                print(f"No face landmarks detected for image {img_pth}. Skipping...")
                continue

            # Extract the face landmarks
            face_landmarks = detection_result.face_landmarks[0]
            coords = [[landmark.x, landmark.y, landmark.z] for landmark in face_landmarks]
            coords = np.array(coords)  # Convert to numpy array

            # Extract features
            features = extract_features(coords)

            # Assign labels based on the face shape directory
            if k == "heart":
                labels.append(0)
            elif k == "oval":
                labels.append(1)
            elif k == "round":
                labels.append(2)
            elif k == "square":
                labels.append(3)
            else:
                print('ERROR')
                break

            faces.append(features)
    
    return np.array(faces), np.array(labels)

In [ ]:
# Paths for training and testing data
train_data_pth = "Face_Shape/training2"
test_data_pth = "Face_Shape/testing2"

# Process the datasets
X_train, y_train = process_data(train_data_pth)
X_test, y_test = process_data(test_data_pth)

# Print dataset shape
print(f"Training dataset shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Testing dataset shape: {X_test.shape}, Testing labels shape: {y_test.shape}")


# Train the RandomForest model
rf_model = RandomForestClassifier(n_estimators=1500, random_state=20)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Save the trained RandomForest model
with open('Best_RandomForest.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
